#  Make Tournament CSV

In [ ]:
from tqdm import tqdm
import pandas as pd
from datetime import datetime
import requests

def get_data(player_id, page, PER_PAGE=490):
    # URL of the GraphQL endpoint
    url = 'https://api.start.gg/gql/alpha'
    
    # The GraphQL query
    query = """
      query PlayerQuery {
      player(id: "{player_id}") {
        id
        gamerTag
        sets(page: {page}, perPage: 490) {
          nodes {
            event {
              tournament {
                slug
                startAt
              }
            }
          }
        }
      }
    }""".replace('{page}', str(page)).replace('{player_id}', str(player_id))
    
    # Headers (if needed, e.g., for authentication)
    headers = {
        'Content-Type': 'application/json',
        'Authorization': 'Bearer <TOKEN>'  # if authentication is needed
    }
    
    # Sending the request
    response = requests.post(url, json={'query': query}, headers=headers)
    
    result = response.json()

    return result


def get_events(result):
    events = result['data']['player']['sets']['nodes']
    events = [event['event']['tournament'] for event in events]
    return events

def get_datetime(timestamp):
    # Convert timestamp to datetime object
    datetime_object = datetime.fromtimestamp(timestamp)
    
    # Format datetime object as a readable string
    readable_date_time = datetime_object.strftime("%Y-%m")
    
    return readable_date_time

def make_df(gamertag, events):
    df = pd.DataFrame(events)
    df = df.drop_duplicates().reset_index(drop=True)
    df.startAt = df.startAt.map(lambda ts: get_datetime(ts))
    df['gamertag'] = gamertag
    reordered_df = df[['gamertag', 'slug', 'startAt']]
    return reordered_df

def get_all_events(player_id):
    page = 1
    all_events = []
    while True:
        # print('page', page)
        result = get_data(player_id, page)
        events = get_events(result)
        if events == []:
            break
        all_events.extend(events)
        page += 1
    return all_events

players = [
    {"gamertag": "Tweek", "id": 15768},
    {"gamertag": "MkLeo", "id": 222927},
    {"gamertag": "Sparg0", "id": 158026},
    {"gamertag": "Light", "id": 158871},
    {"gamertag": "あcola", "id": 2691639},
    {"gamertag": "Glutonny", "id": 6122},
    {"gamertag": "Riddles", "id": 160464},
    {"gamertag": "Tea", "id": 695882},
    {"gamertag": "Kurama", "id": 175623},
]

event_df = pd.DataFrame(columns=['gamertag', 'slug', 'startAt'])
for player in players:
    events = get_all_events(player['id'])
    df = make_df(player['gamertag'], events)
    event_df = pd.concat([event_df, df]).reset_index(drop=True)

event_df.slug = event_df.slug.map(lambda event: event.lstrip('tournament/')) # TODO do upstream
event_df

# Save Tournaments CSV

In [ ]:
event_df.to_csv('tournaments.csv', index=False)

# Find Most Common Tournaments

In [ ]:
def print_tournaments_by_popularity(df, head):
    groups = df.groupby(['slug', 'startAt'])
    for (slug, startAt), _ in list(df.groupby(['slug', 'startAt']).size().sort_values(ascending=False).items())[:head]:
        group = groups.get_group((slug, startAt))
        players = ', '.join(group.gamertag)
        print(startAt, slug, '→', players)

print_tournaments_by_popularity(event_df, head=20)

# Find Tournaments This Year

In [ ]:
this_year_df = event_df[event_df.startAt.str.contains('2024')].reset_index(drop=True)

print_tournaments_by_popularity(this_year_df, head=10)